In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import hackathon.prompts as prompts
from dotenv import load_dotenv
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
import os
import math


# For the agents
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
import langchain.llms as llms
import time
import json
from flask import Flask, request, jsonify

#For the main OPENAI ASSISTANT
import openai as openai
#
load_dotenv()

True

In [4]:
# Constants
DATA_PATH = "/Users/arthur.cruiziat/dev/Qonto_hackathon/data/random_trx_df.csv"
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Read data

In [5]:
data = pd.read_csv(DATA_PATH)
data_filterered = data.copy()

In [6]:
def create_temporal_features(df, date_column):
    """
    Adds temporal features to the DataFrame.

    Parameters:
    df (DataFrame): The input DataFrame with a date or datetime column.
    date_column (str): The name of the column containing the date or datetime.

    Returns:
    DataFrame: The original DataFrame with additional columns for temporal features.
    """

    # Ensure the date column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column])

    # Day of the week (0 = Monday, 6 = Sunday)
    df["day_of_week"] = df[date_column].dt.dayofweek

    # Day of the year
    df["day_of_year"] = df[date_column].dt.dayofyear

    # Week of the year
    df["week_of_year"] = df[date_column].dt.isocalendar().week

    # Week of the month
    df["week_of_month"] = df[date_column].apply(lambda x: math.ceil(x.day / 7.0))

    # Weekday (1 if it's a weekday, 0 if it's a weekend)
    df["is_weekday"] = df[date_column].dt.weekday < 5

    # Month
    df["month"] = df[date_column].dt.month

    # Year
    df["year"] = df[date_column].dt.year

    return df

In [7]:
test = data.copy()
test = create_temporal_features(test, "date")

In [8]:
test.head()

,user,date,amount,trx_category,method,team,receipt,VAT,Status,day_of_week,day_of_year,week_of_year,week_of_month,is_weekday,month,year
0,user_3,2020-11-19,7,Restaurants,taxes,Marketing,Not needed,Missing,Executed,3,324,47,3,True,11,2020
1,user_2,2021-11-10,977,Health,direct_debit,Sales,Missing,Filled in,Declined,2,314,45,2,True,11,2021
2,user_2,2021-03-04,130,Travel,card,Marketing,Not needed,Missing,Executed,3,63,9,1,True,3,2021
3,user_3,2020-11-10,57,Mobile,direct_debit,Finance,Not needed,Missing,Executed,1,315,46,2,True,11,2020
4,user_1,2020-01-20,838,Travel,card,Finance,Attached,Missing,Processing,0,20,4,3,True,1,2020


In [9]:
def prompt_to_filter(prompt, data_to_filter):
    """
    Applies a given prompt to filter a DataFrame using OpenAI's GPT model.

    Args:
    - prompt (str): A prompt describing the filter criteria for the DataFrame.
    - data (DataFrame): The DataFrame to be filtered.
    - OPENAI_API_KEY (str): The API key for OpenAI.

    Returns:
    - DataFrame: The filtered DataFrame based on the response from the OpenAI model.

    Raises:
    - ValueError: If an invalid API key is provided.
    - RuntimeError: If the maximum number of retries is reached.

    This function attempts up to 5 retries in case of transient errors.
    """
    OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

    retry_count = 0
    max_retries = 5
    wait_seconds = 2  # Time to wait between retries

    data = create_temporal_features(data_to_filter, "date")
    while retry_count < max_retries:
        try:
            llm = OpenAI(OPENAI_API_KEY)
            df = SmartDataframe(
                data, config={"llm": llm, "llm_model": "gpt-4-1106-preview"}
            )
            response = df.chat(prompt).convert_dtypes()
            return response
        except Exception as e:
            retry_count += 1
            if retry_count >= max_retries:
                return "The function did not work, please let the user know that we are not able to filter the transactions."
            time.sleep(wait_seconds)

In [10]:
data.head(2)

,user,date,amount,trx_category,method,team,receipt,VAT,Status
0,user_3,2020-11-19,7,Restaurants,taxes,Marketing,Not needed,Missing,Executed
1,user_2,2021-11-10,977,Health,direct_debit,Sales,Missing,Filled in,Declined


In [11]:
test = prompt_to_filter(
    "filter the dataframe to only keep the rows where the amount is greater than 900",
    data,
)

In [12]:
type(test.convert_dtypes())

pandas.core.frame.DataFrame

In [13]:
def dataframe_insights(prompt, data_to_filter):
    """
    Analyzes a given DataFrame with a specific prompt using OpenAI's GPT model.

    Args:
    - prompt (str): A prompt describing the analysis or question for the DataFrame.
    - data (DataFrame): The DataFrame to analyze.
    - OPENAI_API_KEY (str): The API key for OpenAI.

    Returns:
    - str: The response from the OpenAI model.

    Raises:
    - ValueError: If an invalid API key is provided.
    - RuntimeError: If the maximum number of retries is reached.

    This function attempts up to 5 retries in case of transient errors.
    """

    OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
    retry_count = 0
    max_retries = 5
    wait_seconds = 2  # Time to wait between retries

    data = create_temporal_features(data_to_filter, "date")

    while retry_count < max_retries:
        try:
            agent = create_pandas_dataframe_agent(
                ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
                data,
                verbose=True,
                agent_type=AgentType.OPENAI_FUNCTIONS,
                handle_parsing_errors=True,
            )
            response = agent.run(prompt)
            return response

        except Exception as e:
            retry_count += 1
            if retry_count == max_retries:
                return "The function did not work, please let the user know that we are not able to give an answer to this question."
            time.sleep(wait_seconds)

In [14]:
data.head(2)

,user,date,amount,trx_category,method,team,receipt,VAT,Status,day_of_week,day_of_year,week_of_year,week_of_month,is_weekday,month,year
0,user_3,2020-11-19,7,Restaurants,taxes,Marketing,Not needed,Missing,Executed,3,324,47,3,True,11,2020
1,user_2,2021-11-10,977,Health,direct_debit,Sales,Missing,Filled in,Declined,2,314,45,2,True,11,2021


In [31]:
aaa = dataframe_insights(
    "filter the dataframe to only keep the rows where the amount is greater than 900, output a dataframe",
    data,
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df_filtered = df[df['amount'] > 900]\ndf_filtered"}`


       user       date  amount    trx_category        method       team  \
1    user_2 2021-11-10     977          Health  direct_debit      Sales   
19   user_3 2020-05-11     913           Sport          fees    Finance   
29   user_1 2021-06-27     981  Food & Grocery          fees      Sales   
30   user_1 2021-11-29     903             ATM  direct_debit         IT   
40   user_1 2021-05-15     956          Health          card  Marketing   
..      ...        ...     ...             ...           ...        ...   
940  user_2 2020-03-23     998           Train      transfer      Sales   
957  user_1 2020-06-17     960        Shopping  direct_debit         HR   
965  user_1 2020-07-17     910          Mobile          fees         IT   
985  user_3 2020-03-22     902            Fees      transfer    Finance   
996  user_2 2020-11-04     931    

In [33]:
print(aaa)

Here is the filtered dataframe where the amount is greater than 900:

|    | user   | date                |   amount | trx_category   | method       | team      | receipt    | VAT       | Status     |   day_of_week |   day_of_year |   week_of_year |   week_of_month | is_weekday   |   month |   year |
|---:|:-------|:--------------------|---------:|:---------------|:-------------|:----------|:-----------|:----------|:-----------|--------------:|--------------:|---------------:|----------------:|:-------------|--------:|-------:|
|  1 | user_2 | 2021-11-10 00:00:00 |      977 | Health         | direct_debit | Sales     | Missing    | Filled in | Declined   |             2 |           314 |             45 |               2 | True         |      11 |   2021 |
| 19 | user_3 | 2020-05-11 00:00:00 |      913 | Sport          | fees         | Finance   | Attached   | Filled in | Executed   |             0 |           132 |             20 |               2 | True         |       5 |   2020 |
| 

In [16]:
aaa = dataframe_insights(
    "What is the average amount, number of transactions and most frequent trx_category for the user 3 ?",
    data,
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df_user_3 = df[df['user'] == 'user_3']\navg_amount = df_user_3['amount'].mean()\nnum_transactions = df_user_3.shape[0]\nmost_frequent_category = df_user_3['trx_category'].mode()[0]\navg_amount, num_transactions, most_frequent_category"}`


(497.11009174311926, 327, 'Shopping')The average amount for user 3 is 497.11, the number of transactions is 327, and the most frequent transaction category is "Shopping".

> Finished chain.


In [17]:
aaa

'The average amount for user 3 is 497.11, the number of transactions is 327, and the most frequent transaction category is "Shopping".'

In [18]:
data.VAT.value_counts()

Filled in    503
Missing      497
Name: VAT, dtype: int64

# Main LLLM

In [19]:
import openai as openai

In [20]:
print(prompts.assistant_instructions)


You are an assistant tasked with analyzing customer transaction data. Your objective is to swiftly and accurately respond to user inquiries regarding their transaction. There are always 2 things to do : share insights regarding the question - either the actual answer or some insights on the transactions displayed - and filter the relevant transactions.
IF a specific category is not precised in the user query, consider all the categories for the insights and the filter.
You can do that using two functions:

1. `dataframe_insights`: Use this function to derive insights from the data based on the user's question. You can use this function multiple times to get the number of transactions, the total amount and the most common trx_category.

2. `dataframe_filter`: This function allows you to filter the transaction data according to the user's query. Filtering occurs automatically based on the formulated prompt. 
Begin prompts with "Filter" followed by COLUMN NAMES and CONDITIONS considering

In [21]:
# Create or load assistant
def create_assistant(client):
    assistant_file_path = "assistant.json"

    # Load existing assistant if file exists
    if os.path.exists(assistant_file_path):
        with open(assistant_file_path, "r") as file:
            assistant_data = json.load(file)
            assistant_id = assistant_data["assistant_id"]
            print("Loaded existing assistant ID.")
    else:
        print("Creating a new assistant...")
        # Create a new assistant if file does not exist
        assistant = client.beta.assistants.create(
            instructions=prompts.assistant_instructions,
            model="gpt-4-1106-preview",
            tools=[
                # {"type": "retrieval"},
                {
                    "type": "function",
                    "function": {
                        "name": "prompt_to_filter",
                        "description": "Applies a given prompt to filter the transaction dataframe from the user.",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "prompt": {
                                    "type": "string",
                                    "description": "A prompt describing the filter criteria for the DataFrame.",
                                },
                                "data_filtered": {
                                    "type": "string",
                                    "description": "The transaction dataframe to be filtered",
                                },
                            },
                            "required": ["prompt"],
                        },
                    },
                },
                {
                    "type": "function",
                    "function": {
                        "name": "dataframe_insights",
                        "description": "Given a prompt, analyzes the transaction dataframe and outputs insights.",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "prompt": {
                                    "type": "string",
                                    "description": "A prompt describing the analysis or question for the DataFrame.",
                                },
                                "data_filtered": {
                                    "type": "string",
                                    "description": "The transaction dataframe to analyse",
                                },
                            },
                            "required": ["prompt"],
                        },
                    },
                },
            ],
            # file_ids=[file.id],
        )

        # Save assistant ID to file for future runs
        with open(assistant_file_path, "w") as file:
            json.dump({"assistant_id": assistant.id}, file)
            print("Created a new assistant and saved the ID.")

        assistant_id = assistant.id

    return assistant_id

In [22]:
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [23]:
assistant_id = create_assistant(client)

Loaded existing assistant ID.


In [24]:
def start_conversation(client):
    print("Starting a new conversation...")
    thread = client.beta.threads.create()
    print(f"New thread created with ID: {thread.id}")
    return {"thread_id": thread.id}

In [25]:
def chat(thread_id, user_input, data_filterered):
    if not thread_id:
        print("Error: Missing thread_id")
        return jsonify({"error": "Missing thread_id"}), 400

    print(f"Received message: {user_input} for thread ID: {thread_id}")

    # Add the user's message to the thread
    client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=user_input
    )

    # Run the Assistant
    run = client.beta.threads.runs.create(
        thread_id=thread_id, assistant_id=assistant_id
    )

    # Check if the Run requires action (function call)
    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id, run_id=run.id
        )
        # print(f"Run status: {run_status.status}")
        if run_status.status == "completed":
            break
        elif run_status.status == "requires_action":
            # Handle the function call
            for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
                if tool_call.function.name == "dataframe_insights":
                    # Process solar panel calculations
                    arguments = json.loads(tool_call.function.arguments)
                    output = dataframe_insights(arguments["prompt"], data_filterered)
                    client.beta.threads.runs.submit_tool_outputs(
                        thread_id=thread_id,
                        run_id=run.id,
                        tool_outputs=[
                            {
                                "tool_call_id": tool_call.id,
                                "output": json.dumps(jsonify({"result": output})),
                            }
                        ],
                    )
                elif tool_call.function.name == "prompt_to_filter":
                    # Process lead creation
                    arguments = json.loads(tool_call.function.arguments)
                    filtered_dataframe = prompt_to_filter(
                        arguments["prompt"], data_filterered
                    )

                    client.beta.threads.runs.submit_tool_outputs(
                        thread_id=thread_id,
                        run_id=run.id,
                        tool_outputs=[
                            {
                                "tool_call_id": tool_call.id,
                                "output": json.dumps(filtered_dataframe.to_json()),
                            }
                        ],
                    )
            time.sleep(1)  # Wait for a second before checking again

    # Retrieve and return the latest message from the assistant
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    response = messages.data[0].content[0].text.value

    print(f"Assistant response: {response}")
    return {"response": response, "data_filterered": filtered_dataframe}

In [26]:
thread = start_conversation(client)
thread["thread_id"]

Starting a new conversation...
New thread created with ID: thread_ZwP9I2Af4aGWCtQujnC66Ctl


'thread_ZwP9I2Af4aGWCtQujnC66Ctl'

In [27]:
response = chat(
    thread["thread_id"],
    "Show me marketing weekend transactions",
    data_filterered.query("user == 'user_3'").drop(columns=["user"]),
)

Received message: Show me marketing weekend transactions for thread ID: thread_ZwP9I2Af4aGWCtQujnC66Ctl


KeyboardInterrupt: 

In [ ]:
response["response"]

"There are 264 transactions that occurred in the year 2021.\n\nHere are a few key points about these transactions:\n- The average transaction amount is 512.71\n- The most prevalent transaction categories include 'Transfer', 'Sport', 'Health', 'Finance', 'Shopping', and 'Food & Grocery' with multiple transactions each."

In [ ]:
response = chat(
    thread["thread_id"],
    "Show me all transactions above 800 for the marketingt team",
    data_filterered.query("user == 'user_3'").drop(columns=["user"]),
)

Received message: Show me all transactions above 800 for the marketingt team for thread ID: thread_1M3JKa8nn87y59DLO6UuvUIC
Assistant response: There are 12 transactions for the Marketing team with amounts greater than 800.

Here are a few key points about these transactions:
- The average transaction amount is 854.75
- The most prevalent transaction category is 'Transfer', 'Hotel', 'ATM', 'Fees', and 'Travel' with 2 transactions each.
- The most common method of payment is 'card', 'taxes', 'fees', and 'direct_debit'.


In [ ]:
print(response["response"])

There are 12 transactions for the Marketing team with amounts greater than 800.

Here are a few key points about these transactions:
- The average transaction amount is 854.75
- The most prevalent transaction category is 'Transfer', 'Hotel', 'ATM', 'Fees', and 'Travel' with 2 transactions each.
- The most common method of payment is 'card', 'taxes', 'fees', and 'direct_debit'.


In [ ]:
response["data_filterered"]

,date,amount,trx_category,method,team,receipt,VAT,Status,day_of_week,day_of_year,week_of_year,week_of_month,is_weekday,month,year
93,2021-04-03,921,Fees,card,Marketing,Missing,Filled in,Declined,5,93,13,1,False,4,2021
209,2021-12-25,892,Mobile,transfer,Marketing,Not needed,Missing,Declined,5,359,51,4,False,12,2021
214,2021-01-12,927,Shopping,fees,Marketing,Not needed,Filled in,Declined,1,12,2,2,True,1,2021
265,2021-08-30,997,Hotel,card,Marketing,Attached,Missing,Declined,0,242,35,5,True,8,2021
280,2021-07-05,868,Transfer,card,Marketing,Missing,Missing,Declined,0,186,27,1,True,7,2021
293,2020-05-03,804,Hotel,direct_debit,Marketing,Attached,Filled in,Executed,6,124,18,1,False,5,2020
677,2021-09-07,857,ATM,direct_debit,Marketing,Missing,Missing,Executed,1,250,36,1,True,9,2021
754,2020-11-29,824,Travel,taxes,Marketing,Missing,Missing,Processing,6,334,48,5,False,11,2020
762,2021-06-17,812,ATM,fees,Marketing,Missing,Missing,Processing,3,168,24,3,True,6,2021
777,2021-02-02,813,Travel,fees,Marketing,Attached,Missing,Processing,1,33,5,1,True,2,2021


In [ ]:
data_filterered

,user,date,amount,trx_category,method,team,receipt,VAT,Status
0,user_3,2020-11-19,7,Restaurants,taxes,Marketing,Not needed,Missing,Executed
1,user_2,2021-11-10,977,Health,direct_debit,Sales,Missing,Filled in,Declined
2,user_2,2021-03-04,130,Travel,card,Marketing,Not needed,Missing,Executed
3,user_3,2020-11-10,57,Mobile,direct_debit,Finance,Not needed,Missing,Executed
4,user_1,2020-01-20,838,Travel,card,Finance,Attached,Missing,Processing
...,...,...,...,...,...,...,...,...,...
995,user_1,2020-09-20,94,Food & Grocery,transfer,Marketing,Missing,Missing,Processing
996,user_2,2020-11-04,931,Taxi,transfer,Finance,Not needed,Filled in,Executed
997,user_3,2020-12-30,401,Train,card,Finance,Missing,Missing,Executed
998,user_3,2020-06-07,850,ATM,direct_debit,Finance,Missing,Filled in,Executed


In [ ]:
response = chat(
    thread["thread_id"],
    "show 2021 marketing transactions for restaurants",
    data_filterered.query("user == 'user_3'").drop(columns=["user"]),
)

Received message: show 2021 marketing transactions for restaurants for thread ID: thread_uOFnwnNouVJ7a0F9hBurfqMW
Assistant response: There was only one transaction for Restaurants by the Marketing team in 2021. The amount was $448, and it was a direct debit with the receipt status as Missing and VAT status as Filled in. The transaction is still Processing.


In [ ]:
response["data_filterered"]

,date,amount,trx_category,method,team,receipt,VAT,Status,day_of_week,day_of_year,week_of_year,week_of_month,is_weekday,month,year


In [ ]:
data_filterered.groupby(["user", "team"]).count()

date  amount  trx_category  method  receipt  VAT  Status
user   team                                                               
user_1 Finance      63      63            63      63       63   63      63
       HR           86      86            86      86       86   86      86
       IT           70      70            70      70       70   70      70
       Marketing    84      84            84      84       84   84      84
       Sales        61      61            61      61       61   61      61
user_2 Finance      47      47            47      47       47   47      47
       HR           53      53            53      53       53   53      53
       IT           71      71            71      71       71   71      71
       Marketing    73      73            73      73       73   73      73
       Sales        65      65            65      65       65   65      65
user_3 Finance      72      72            72      72       72   72      72
       HR           62      62            62      62       62   62      62
       IT           61      61            61      61       61   61      61
       Marketing    63      63            63      63       63   63      63
       Sales        69      69            69      69       69   69      69

In [ ]:
response["data_filterered"]

,date,amount,trx_category,method,team,receipt,VAT,Status,day_of_week,day_of_year,week_of_year,week_of_month,is_weekday,month,year
17,2021-07-03,539,Food & Grocery,direct_debit,Finance,Attached,Missing,Declined,5,184,26,1,False,7,2021
35,2021-01-07,114,Finance,taxes,Finance,Attached,Filled in,Processing,3,7,1,1,True,1,2021
47,2020-02-02,424,Train,fees,HR,Not needed,Missing,Declined,6,33,5,1,False,2,2020
81,2021-09-01,726,Finance,fees,Marketing,Not needed,Missing,Declined,2,244,35,1,True,9,2021
84,2021-02-05,353,Taxi,taxes,Sales,Not needed,Missing,Declined,4,36,5,1,True,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,2021-01-03,486,Food & Grocery,transfer,Marketing,Missing,Filled in,Executed,6,3,53,1,False,1,2021
950,2021-08-02,280,Health,transfer,Sales,Not needed,Filled in,Processing,0,214,31,1,True,8,2021
966,2020-01-04,553,Food & Grocery,fees,HR,Not needed,Missing,Processing,5,4,1,1,False,1,2020
983,2020-09-03,207,Health,direct_debit,Sales,Missing,Missing,Processing,3,247,36,1,True,9,2020


In [ ]:
data_filterered.groupby("team").sum()

/var/folders/0g/vqgn49092qvcqnw14pz_0jsc0000gn/T/ipykernel_6148/677782899.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_filterered.groupby("team").sum()


,amount
team,
Finance,93940
HR,93926
IT,103929
Marketing,109313
Sales,94907
